In [1]:
import numpy as np
import pandas as pd        # For loading and processing the dataset
import tensorflow as tf    # Of course, we need TensorFlow.
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('FF/input/train.csv')
df_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_train = df_train.drop(['PassengerId','Name','Ticket', 'Cabin'], axis=1)

In [4]:
df_train['Sex'] = df_train['Sex'].map({'female':0, 'male':1}).astype(int) 

In [5]:
# We replace 'Embarked' by three dummy variables 'Embarked_S', 'Embarked_C', and 'Embarked Q',
# which are 1 if the person embarked there, and 0 otherwise.
df_train = pd.concat([df_train, pd.get_dummies(df_train['Embarked'], prefix='Embarked')], axis=1)
df_train = df_train.drop('Embarked', axis=1)

In [6]:
# We normalize the age and the fare by subtracting their mean and dividing by the standard deviation
age_mean = df_train['Age'].mean()
age_std = df_train['Age'].std()
df_train['Age'] = (df_train['Age'] - age_mean) / age_std

fare_mean = df_train['Fare'].mean()
fare_std = df_train['Fare'].std()
df_train['Fare'] = (df_train['Fare'] - fare_mean) / fare_std

In [7]:
# In many cases, the 'Age' is missing - which can cause problems. Let's look how bad it is:
print("Number of missing 'Age' values: {:d}".format(df_train['Age'].isnull().sum()))

# A simple method to handle these missing values is to replace them by the mean age.
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())

Number of missing 'Age' values: 177


In [8]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,-0.530005,1,0,-0.502163,0,0,1
1,1,1,0,0.571430,1,0,0.786404,1,0,0
2,1,3,0,-0.254646,0,0,-0.488580,0,0,1
3,1,1,0,0.364911,1,0,0.420494,0,0,1
4,0,3,1,0.364911,0,0,-0.486064,0,0,1


In [9]:
# Finally, we convert the Pandas dataframe to a NumPy array, and split it into a training and test set
X_train = df_train.drop('Survived', axis=1).to_numpy()
y_train = df_train['Survived'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

np.savetxt('test_seen.out', X_test, header="{},{}".format(*X_test.shape), delimiter=",")
np.savetxt('train.out', X_train, header="{},{}".format(*X_train.shape), delimiter=",")

In [10]:
# We'll build a classifier with two classes: "survived" and "didn't survive",
# so we create the according labels
# This is taken from https://www.kaggle.com/klepacz/titanic/tensor-flow
labels_train = (np.arange(2) == y_train[:,None]).astype(np.float32)
labels_test = (np.arange(2) == y_test[:,None]).astype(np.float32)

In [11]:
tf.compat.v1.disable_eager_execution()

inputs = tf.compat.v1.placeholder(tf.float32, shape=(None, X_train.shape[1]), name='inputs')
label = tf.compat.v1.placeholder(tf.float32, shape=(None, 2), name='labels')

In [12]:
# First layer
hid1_size = 128
w1 = tf.Variable(tf.random.normal([hid1_size, X_train.shape[1]], stddev=0.01), name='w1')
b1 = tf.Variable(tf.constant(0.1, shape=(hid1_size, 1)), name='b1')
y1 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(w1, tf.transpose(a=inputs)), b1)), rate=1 - (0.5))

In [13]:
# Second layer
hid2_size = 256
w2 = tf.Variable(tf.random.normal([hid2_size, hid1_size], stddev=0.01), name='w2')
b2 = tf.Variable(tf.constant(0.1, shape=(hid2_size, 1)), name='b2')
y2 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(w2, y1), b2)), rate=1 - (0.5))

In [14]:
# Output layer
wo = tf.Variable(tf.random.normal([2, hid2_size], stddev=0.01), name='wo')
bo = tf.Variable(tf.random.normal([2, 1]), name='bo')
yo = tf.transpose(a=tf.add(tf.matmul(wo, y2), bo))

In [15]:
# Loss function and optimizer
lr = tf.compat.v1.placeholder(tf.float32, shape=(), name='learning_rate')
loss = tf.reduce_mean(input_tensor=tf.nn.softmax_cross_entropy_with_logits(logits=yo, labels=tf.stop_gradient(label)))
optimizer = tf.compat.v1.train.GradientDescentOptimizer(lr).minimize(loss)

In [16]:
# Prediction
pred = tf.nn.softmax(yo)
pred_label = tf.argmax(input=pred, axis=1)
correct_prediction = tf.equal(tf.argmax(input=pred, axis=1), tf.argmax(input=label, axis=1))
accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, "float"))

In [17]:
# Create operation which will initialize all variables
init = tf.compat.v1.global_variables_initializer()

# Configure GPU not to use all memory
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

# Start a new tensorflow session and initialize variables
sess = tf.compat.v1.InteractiveSession(config=config)
sess.run(init)

In [18]:
w1_eval = w1.eval(session=sess)
np.savetxt('w1.out', w1_eval, header="{},{}".format(*w1_eval.shape), delimiter=",")

In [19]:
writer = tf.compat.v1.summary.FileWriter('logs', sess.graph)

# This is the main training loop: we train for 50 epochs with a learning rate of 0.05 and another 
# 50 epochs with a smaller learning rate of 0.01
for learning_rate in [0.05, 0.01]:
    for epoch in range(50):
        avg_cost = 0.0

        # For each epoch, we go through all the samples we have.
        for i in range(X_train.shape[0]):
            # Finally, this is where the magic happens: run our optimizer, feed the current example into X and the current target into Y
            _, c = sess.run([optimizer, loss], feed_dict={lr:learning_rate, 
                                                          inputs: X_train[i, None],
                                                          label: labels_train[i, None]})
            avg_cost += c
        avg_cost /= X_train.shape[0]    

        # Print the cost in this epcho to the console.
        if epoch % 10 == 0:
            print("Epoch: {:3d}    Train Cost: {:.4f}".format(epoch, avg_cost))

Epoch:   0    Train Cost: 0.6564
Epoch:  10    Train Cost: 0.5041
Epoch:  20    Train Cost: 0.4844
Epoch:  30    Train Cost: 0.5090
Epoch:  40    Train Cost: 0.4700
Epoch:   0    Train Cost: 0.4797
Epoch:  10    Train Cost: 0.4481
Epoch:  20    Train Cost: 0.4187
Epoch:  30    Train Cost: 0.4158
Epoch:  40    Train Cost: 0.3922


In [20]:
acc_train = accuracy.eval(feed_dict={inputs: X_train, label: labels_train})
print("Train accuracy: {:3.2f}%".format(acc_train*100.0))
acc_test = accuracy.eval(feed_dict={inputs: X_test, label: labels_test})
print("Test accuracy:  {:3.2f}%".format(acc_test*100.0))

Train accuracy: 81.74%
Test accuracy:  83.24%


In [21]:
op = yo.eval(feed_dict={inputs: X_test})
#op

In [22]:
for i in range(len(op)):
    op[i] = np.exp(op[i]) / np.sum(np.exp(op[i]))
#op

In [23]:
correct = 0
for i in range(len(op)):
    if labels_test[i][0] > labels_test[i][1]:
        if op[i][0] > op[i][1]:
            correct += 1
    elif labels_test[i][0] < labels_test[i][1]:
        if op[i][0] < op[i][1]:
            correct += 1
print("Acc: {}/{}".format(correct, len(op)))

Acc: 144/179


In [24]:
#op

In [25]:
w1_eval = w1.eval(session=sess)
np.savetxt('w1.out', w1_eval, header="{},{}".format(*w1_eval.shape), delimiter=",")

w2_eval = w2.eval(session=sess)
np.savetxt('w2.out', w2_eval, header="{},{}".format(*w2_eval.shape), delimiter=",")

wo_eval = wo.eval(session=sess)
np.savetxt('wo.out', wo_eval, header="{},{}".format(*wo_eval.shape), delimiter=",")

b1_eval = b1.eval(session=sess)
np.savetxt('b1.out', b1_eval, header="{},{}".format(*b1_eval.shape), delimiter=",")

b2_eval = b2.eval(session=sess)
np.savetxt('b2.out', b2_eval, header="{},{}".format(*b2_eval.shape), delimiter=",")

bo_eval = bo.eval(session=sess)
np.savetxt('bo.out', bo_eval, header="{},{}".format(*bo_eval.shape), delimiter=",")

In [26]:
#w1.eval(session=sess)

In [27]:
df_test = pd.read_csv('FF/input/test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [28]:
# Do all pre-processing steps as above
df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df_test['Sex'] = df_test['Sex'].map({'female':0, 'male':1}).astype(int)
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')], axis=1)
df_test = df_test.drop('Embarked', axis=1)
df_test['Age'] = (df_test['Age'] - age_mean) / age_std
df_test['Fare'] = (df_test['Fare'] - fare_mean) / fare_std
df_test.head()
X_test = df_test.drop('PassengerId', axis=1).to_numpy()
np.savetxt('test_unseen.out', X_test, header="{},{}".format(*X_test.shape), delimiter=",")

In [29]:
for i in range(X_test.shape[0]):
    df_test.loc[i, 'Survived'] = sess.run(pred_label, feed_dict={inputs: X_test[i, None]}).squeeze()

writer.close()

In [30]:
sess.close()

In [31]:
output = pd.DataFrame()
output['PassengerId'] = df_test['PassengerId']
output['Survived'] = df_test['Survived'].astype(int)
output.to_csv('FF/output/prediction.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [32]:
np.savetxt('labels_test_seen.out', labels_test, header="{},{}".format(*labels_test.shape), delimiter=",")
labels_test

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.